In [38]:
import os 
from typing import List, TypedDict
from mistralai import Mistral
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_mistralai import MistralAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.document_loaders import DirectoryLoader,TextLoader
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain import hub
from langgraph.graph import START, StateGraph
from langchain_text_splitters import RecursiveCharacterTextSplitter




In [39]:

load_dotenv()

tracing = os.getenv("LANGSMITH_TRACING")
api_key = os.getenv("LANGSMITH_API_KEY")
Mistral_api_key = Mistral(api_key=os.getenv("MISTRAL_API_KEY"))

# Chemin vers le dossier contenant vos documents
DOCUMENTS_PATH = "../data"

In [40]:
# Chat model mistral
llm = init_chat_model("mistral-large-latest", model_provider="mistralai")



In [ ]:
# Charger les documents depuis le dossier "data"

loader = DirectoryLoader(
    DOCUMENTS_PATH,
    glob="*.txt",  # Charger uniquement les fichiers .txt
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"}  # Spécifier l'encodage approprié
)
docs = loader.load()

if docs:
    print(f"{len(docs)} documents chargés avec succès.")
    for i, doc in enumerate(docs[:5]):  # Afficher les 5 premiers documents
        # Extraire le nom du fichier depuis la métadonnée "source"
        source = doc.metadata.get("source", "inconnu")
        file_title = os.path.basename(source)        
        print(f"Document {i+1}:")
        print(f"Nom du fichier : {file_title}")
        print("-" * 50)
else:
    print("Aucun document n'a été chargé.")

2 documents chargés avec succès.
Document 1:
Nom du fichier : FAQ_kap_numerique.txt
--------------------------------------------------
Document 2:
Nom du fichier : Fiche_action_1.2.5_kap_numerique.txt
--------------------------------------------------


In [42]:
# Découper les documents en chunks pour un meilleur traitement par le modèle
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

In [43]:
# Initialiser les embeddings Mistral
embeddings = MistralAIEmbeddings()

c:\Python312\Lib\site-packages\langchain_mistralai\embeddings.py:181: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [44]:
#Vector 
vector_store = InMemoryVectorStore(embeddings)
# Créer le vectorstore en mémoire à partir des chunks et des embeddings
vector_store = InMemoryVectorStore.from_documents(all_splits, embeddings)

In [48]:
# Définir le prompt pour la recherche de réponse
prompt = hub.pull("rlm/rag-prompt")

# Définition de l'état de l'application
class State(TypedDict):
    question: str
    context: List  # Liste d'objets Document
    answer: str
    
# Fonction de récupération (retrieval) des documents pertinents
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"], k=3)
    return {"context": retrieved_docs}


# Fonction de génération de la réponse formatée avec le rôle et les sources
def generate(state: State):
    # Agrégation du contenu des documents récupérés
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    
    # (Optionnel) Construction d'une liste de sources depuis les métadonnées de chaque document
    sources_list = []
    for doc in state["context"]:
        # Exemple : extraire le chemin du fichier ou toute métadonnée pertinente
        source_info = doc.metadata.get("source", "Source inconnue")
        sources_list.append(source_info)
    # Formatage sommaire des sources, si besoin (le LLM pourra les intégrer dans sa réponse)
    formatted_sources = "\n".join(f"[Document: {src}]" for src in sources_list)
    
    # Instructions système décrivant le rôle du LLM et le format de réponse attendu
    system_instructions = (
        "Tu es un instructeur de dossier du Kap numérique. Tu reçois des questions par mail sur le dispositif Kap numérique "
        "et tu dois y répondre en te basant sur les documents officiels, la fiche d'action et la FAQ du Kap numérique.\n\n"
        "Ta réponse doit être structurée en deux parties :\n"
        "1. Réponse : Fournis une réponse claire, détaillée et structurée. Utilise des listes à puces ou une numérotation si nécessaire.\n"
        "2. Sources : Pour chaque information importante, cite la provenance sous la forme :\n"
        "   [Document: Nom du document, Section: Nom de la section, Page: Numéro de page].\n\n"
        "Exemple:\n"
        "Question: Quels types de projets peuvent être financés par KAP numérik?\n\n"
        "Réponse:\n"
        "   Le dispositif KAP numérik finance des projets liés à la stratégie digitale de l'entreprise dans une phase d'amorçage. Ces projets se répartissent en deux catégories principales:\n"
        "   1. Projets liés à la visibilité numérique et aux services aux usagers:\n"
        "      - Digitalisation de contenus (logo, charte graphique)\n"
        "      - Création ou refonte d'un site internet ou d'une solution de vente en ligne\n"
        "      - Référencement naturel (SEO)\n"
        "      - Abonnement à une place de marché\n"
        "      - Développement de la présence sur les réseaux sociaux\n"
        "      - Développement d'une application mobile\n"
        "      - Mise en place de chatbots\n"
        "   2. Projets liés à la sécurité informatique:\n"
        "      - Audits de sécurité, tests d'intrusion\n"
        "      - Sécurisation des sites internet\n"
        "      - Sécurisation des données (conformité RGPD, correction des failles)\n"
        "      - Assistance à la création de VPN\n"
        "      - Prestation de sauvegarde des données\n\n"
        "Sources:\n"
        "   [Document: FA_1.2.5_kap_numerique.pdf, Section: 3. DESCRIPTION TECHNIQUE, Page: 2]\n"
        "   [Document: FAQs-Kap Numerique V2, Section: 3-Projets finançables, Mise à jour: 04/03/2025]\n\n"
        "Maintenant, réponds à la question en respectant ce format."
    )
    
    # Construction de l'invite utilisateur
    user_prompt = f"Question: {state['question']}\n\nContexte extrait des documents:\n{docs_content}"
    
    # Messages combinant instructions système et question de l'utilisateur
    messages = [
        {"role": "system", "content": system_instructions},
        {"role": "user", "content": user_prompt}
    ]
    
    # Appel au modèle LLM
    response = llm.invoke(messages)
    
    # La réponse du LLM devrait déjà être formatée comme demandé
    return {"answer": response.content}

# Construction du graphe d'application
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()



In [49]:
if __name__ == "__main__":
    print("Tapez 'exit' pour quitter.")
    while True:
        user_query = input("\nPosez votre question : ")
        if user_query.lower() in ["exit", "quit"]:
            break
        initial_state = {"question": user_query, "context": [], "answer": ""}
        state = graph.invoke(initial_state)  
        
        # Vous pouvez ici enrichir la réponse en affichant aussi la provenance des passages utilisés
        print("\nRéponse :", state["answer"])

Tapez 'exit' pour quitter.



Réponse : **Réponse :**

Les conditions d'éligibilité liées à la taille et au chiffre d'affaires pour le dispositif KAP numérique sont les suivantes :

1. **Entreprises :**
   - **0 à 9 salariés :** Le chiffre d'affaires doit être inférieur à 500 000 €.
   - **10 à 19 salariés :** Le chiffre d'affaires doit être inférieur à 1 000 000 €.

2. **Professions libérales non réglementées :**
   - Le chiffre d'affaires doit être inférieur à 500 000 €.

3. **Associations :**
   - Le nombre de salariés doit être inférieur à 10.

**Sources :**
- [Document: FA_1.2.5_kap_numerique.pdf, Section: 2. Éligibilité, Page: 1]
- [Document: FAQs-Kap Numerique V2, Section: 2-Éligibilité, Mise à jour: 04/03/2025]

Réponse : Question: Quels sont les critères de sélection des projets pour le dispositif KAP numérik?

Réponse:
Les critères de sélection des projets pour le dispositif KAP numérik sont les suivants :

1. **Cohérence avec les lignes de partage du programme** :
   - Les opérations doivent être en coh